In [1]:
import requests

In [4]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the land of code, a tale unfolds,\nOf a function recursive, its magic untold.\nLike a mirror reflecting its own gentle gaze,\nRepeating itself in a wondrous maze.\n\nIt calls upon itself, again and again,\nA loop of enchantment, a never-ending chain.\nWith each iteration, a new journey untold,\nUnraveling mysteries, breaking barriers bold.\n\nThrough depths of logic, it travels deep,\nIn search of answers it tirelessly peep.\nLike a phoenix rising from ashes of yore,\nRecursion explores, seeking more and more.\n\nA loop within a loop, a dance of time,\nIn the realm of programming, it's truly sublime.\nFor in its elegance, a beauty resides,\nA pattern repeated, where wisdom abides.\n\nSo heed the call of recursion's song,\nIn the world of coding, where dreams belong.\nAn infinite loop with a purpose defined,\nIn the tapestry of logic, its brilliance enshrined.", role='assistant', function_call=None, tool_calls=None)
